In [ ]:
# for each day per fire save as new layer and cumulative layer 
# Create Buffer OUTSIDE_ONLY, ALL
# [500m, 1km, 5km]
# merge 3 buffers into one : FireName_JulianDay_Year.shp 
# Zonal Stats for all variables 

In [1]:
import pandas as pd
import urllib
import time
from datetime import datetime, date, time, timedelta
import io
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os 
import matplotlib.pyplot as plt
import arcpy
from arcpy import env
from arcpy.sa import *
arcpy.overwriteoutput = True

env.workspace = "F:\DriversFireProject\TEMP"

In [2]:
# create function that finds all shapefiles in root 

def lstFiles(rootPath, ext):
    emptyList = []
    root = rootPath
    for path, subdirs, files in os.walk(root):
        for names in files: 
            if names.endswith(ext) and not names.startswith("._"):
                emptyList.append(path + '\\' + names)
    return(emptyList)

In [5]:
Finalmerged = pd.read_csv(r'F:\DriversFireProject\NaturalNeighborResults\FinalDF.csv', index_col=0)
Finalmerged.head()

,Fire,Area(ha),Year,JD_A,JD_B,PT_A,PT_B,direction,distance,Total Days,Total Area(ha),Ratio,cumsum,FID_ca_eco,EcoRegion
0,Bagleycomplex,283.020268,2012,0,231,"(40.9909947772, -122.072186413)","(41.0053171596, -122.043491753)",56.512489,2.887099,23,18603.344336,0.015213,0.015213,7,Klamath Mountains
1,Bagleycomplex,697.831159,2012,231,232,"(41.0053171596, -122.043491753)","(41.0090568338, -122.04037631799999)",32.155006,0.491179,23,18603.344336,0.037511,0.052724,7,Klamath Mountains
2,Bagleycomplex,813.693831,2012,232,233,"(41.0090568338, -122.04037631799999)","(41.0078275693, -122.03882670799999)",136.430234,0.188654,23,18603.344336,0.043739,0.096464,7,Klamath Mountains
3,Bagleycomplex,858.176194,2012,233,234,"(41.0078275693, -122.03882670799999)","(41.0066959359, -122.042133041)",245.604318,0.304639,23,18603.344336,0.046130,0.142594,7,Klamath Mountains
4,Bagleycomplex,957.232491,2012,234,235,"(41.0066959359, -122.042133041)","(41.0064471794, -122.045402632)",264.243995,0.275747,23,18603.344336,0.051455,0.194049,7,Klamath Mountains


In [7]:
NAT_SHP = lstFiles(r'F:\DriversFireProject\NaturalNeighborResults\Daily\Dis_Shapefile', ".shp")
NAT_SHP

['F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\Dis_Shapefile\\2012\\Bagleycomplex\\Bagleycomplex_2012_NAT.shp',
 'F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\Dis_Shapefile\\2012\\Banner\\Banner_2012_NAT.shp',
 'F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\Dis_Shapefile\\2012\\Chihuahua2\\Chihuahua2_2012_NAT.shp',
 'F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\Dis_Shapefile\\2012\\Chips\\Chips_2012_NAT.shp',
 'F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\Dis_Shapefile\\2012\\Dale\\Dale_2012_NAT.shp',
 'F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\Dis_Shapefile\\2012\\Fortcomplex\\Fortcomplex_2012_NAT.shp',
 'F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\Dis_Shapefile\\2012\\Fortcomplexgoff\\Fortcomplexgoff_2012_NAT.shp',
 'F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\Dis_Shapefile\\2012\\George\\George_2012_NAT.shp',
 'F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\Dis_Shapefile\\2012\\Highland\\High

In [5]:
# create buffers at ignition point locations to predict first day

ignitionPoints = lstFiles(r'F:\DriversFireProject\Ignition\Centroid', '.shp')
rootpath = r'F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\'

for ignition in ignitionPoints:
    nm = ignition.split("\\")
    outName = nm[-1]
    nmm = outName.split("_")
    fr = nmm[0]
    yr = nmm[1]
    
    buff_fireFolder = rootpath + 'MergedBuff\\' + yr + "\\" + fr
    
    print(ignition)
    try: 
        Buffer_200M = arcpy.Buffer_analysis(ignition, rootpath + 'BufferLayers\\' + fr + "_" + yr + "_0_200m.shp",
                              "200 METERS") 
        Buffer_1KM = arcpy.Buffer_analysis(ignition, rootpath + 'BufferLayers\\' + fr + "_" + yr + "_0_1Km.shp",
                              "1000 METERS")     
        Buffer_5KM = arcpy.Buffer_analysis(ignition, rootpath + 'BufferLayers\\' + fr + "_" + yr + "_0_5Km.shp",
                              "5000 METERS")    
        arcpy.Merge_management([Buffer_200M, Buffer_1KM, Buffer_5KM], buff_fireFolder + "\\" + fr + "_" + yr + "_0_NAT.shp")
    except: 
        print("error: " + fr + yr)
        continue

F:\DriversFireProject\Ignition\Centroid\Adobe_2017_VII_M6.shp
error: Adobe2017
F:\DriversFireProject\Ignition\Centroid\Alamo_2017_VII_M6.shp
error: Alamo2017
F:\DriversFireProject\Ignition\Centroid\Alder_2018_VII_M6.shp
error: Alder2018
F:\DriversFireProject\Ignition\Centroid\American_2013_VII_M6.shp
error: American2013
F:\DriversFireProject\Ignition\Centroid\Aspen_2013_VII_M6.shp
error: Aspen2013
F:\DriversFireProject\Ignition\Centroid\Atlas_2017_VII_M6.shp
error: Atlas2017
F:\DriversFireProject\Ignition\Centroid\Bagleycomplex_2012_VII_M6.shp
error: Bagleycomplex2012
F:\DriversFireProject\Ignition\Centroid\Banner_2012_VII_M6.shp
error: Banner2012
F:\DriversFireProject\Ignition\Centroid\Basilonecomplexcombat_2014_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Basilonecomplexpulgas_2014_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Basilonecomplextomahawk_2014_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Berry_2017_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Bluecu

F:\DriversFireProject\Ignition\Centroid\Highland_2012_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Highway_2017_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Hirz_2018_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Holcomb_2017_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Holy_2018_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Hudson_2017_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Humboldtcomplexsteelhead_2015_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Humboldtcomplex_2015_VII_M6.shp
error: Humboldtcomplex2015
F:\DriversFireProject\Ignition\Centroid\Humboldtlca_2015_VII_M6.shp
error: Humboldtlca2015
F:\DriversFireProject\Ignition\Centroid\Islandknf_2017_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Jacobson_2016_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Jawbonecomplex_2012_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Jayne_2017_VII_M6.shp
error: Jayne2017
F:\DriversFireProject\Ignition\Centroid\Jerusalem_2015_VII_M6.shp
F:\D

F:\DriversFireProject\Ignition\Centroid\Salmonrivercomplexboulder_2013_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Salmonrivercomplexshelly_2013_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Sandiegocomplexcocos_2014_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Sand_2019_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Sangabrielcomplexfish_2016_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Sangabrielcomplexreservoir_2016_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Sawmill_2016_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Scotts_2012_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Sherpa_2016_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Shirley_2014_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Shockey_2012_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Shucreek_2018_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Silver_2013_VII_M6.shp
F:\DriversFireProject\Ignition\Centroid\Sitescomplex_2012_VII_M6.shp
F:\DriversFireProject\I

In [52]:
arcpy.overwriteoutput = True

errorFires = []
for index, row in Finalmerged[169:].iterrows():
    fr = row.Fire
    yr = str(row.Year)
    JD = str(row.JD_B)
    print(index, fr, JD)
    rootpath = r'F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\'
    AF = rootpath + 'Dis_Shapefile\\' + yr + "\\" + fr + "\\" + fr + "_" + yr + "_NAT.shp"
    SQL = """ gridcode = {}""".format(JD)
    sel = arcpy.SelectLayerByAttribute_management(AF, 'NEW_SELECTION', SQL)
    # Write the selected features to a new featureclass
    
    yrfolder =  rootpath + 'DayLayer\\' + yr
    if not os.path.exists(yrfolder):
        os.makedirs(yrfolder)
    fireFolder = yrfolder + "\\" + fr
    if not os.path.exists(fireFolder):
        os.makedirs(fireFolder)
        
    outName = fr + "_" + yr + "_" + str(JD) +"_NAT.shp"
    perimeter = arcpy.CopyFeatures_management(sel, fireFolder + "\\" + outName)
    SHPLIST = lstFiles(fireFolder, '.shp')
    
    cumuyrfolder =  rootpath + 'CumuDayLayer\\' + yr
    if not os.path.exists(cumuyrfolder):
        os.makedirs(cumuyrfolder)
    cumufireFolder = cumuyrfolder + "\\" + fr
    if not os.path.exists(cumufireFolder):
        os.makedirs(cumufireFolder)
            
    buff_yrfolder =  rootpath + 'MergedBuff\\' + yr
    if not os.path.exists(buff_yrfolder):
        os.makedirs(buff_yrfolder)
    buff_fireFolder = buff_yrfolder + "\\" + fr
    if not os.path.exists(buff_fireFolder):
        os.makedirs(buff_fireFolder)
    try: 
        if len(SHPLIST) > 1: 
            clipper = arcpy.Merge_management(SHPLIST, cumufireFolder + "\\" + outName)

            Buffer_200M = arcpy.Buffer_analysis(perimeter, rootpath + 'BufferLayers\\' + fr + "_" + yr + "_" + str(JD) +"_200m.shp",
                                  "200 METERS", "OUTSIDE_ONLY", "ROUND", "ALL")
            Erased_Buffer_200M = arcpy.Erase_analysis(Buffer_200M, clipper, rootpath + 'ClippedBufferLayers\\' + fr + "_" + str(JD) +"_200m.shp")

            Buffer_1KM =arcpy.Buffer_analysis(perimeter, rootpath + 'BufferLayers\\' + fr + "_" + yr + "_" + str(JD) +"_1Km.shp",
                                  "1000 METERS", "OUTSIDE_ONLY", "ROUND", "ALL")
            Erased_Buffer_1KM = arcpy.Erase_analysis(Buffer_1KM, clipper, rootpath + 'ClippedBufferLayers\\' + fr + "_" + str(JD) +"_1Km.shp")

            Buffer_5KM =arcpy.Buffer_analysis(perimeter, rootpath + 'BufferLayers\\' + fr + "_" + yr + "_" + str(JD) +"_5Km.shp",
                                  "5000 METERS", "OUTSIDE_ONLY", "ROUND", "ALL")
            Erased_Buffer_5KM = arcpy.Erase_analysis(Buffer_5KM, clipper, rootpath + 'ClippedBufferLayers\\' + fr + "_" + str(JD) +"_5Km.shp")

            arcpy.Merge_management([Erased_Buffer_200M, Erased_Buffer_1KM, Erased_Buffer_5KM], buff_fireFolder + "\\" + outName)

        else: 
            arcpy.CopyFeatures_management(SHPLIST[0], cumufireFolder + "\\" + outName)

            Buffer_200M = arcpy.Buffer_analysis(perimeter, rootpath + 'BufferLayers\\' + fr + "_" + yr + "_" + str(JD) +"_200m.shp",
                                  "200 METERS", "OUTSIDE_ONLY", "ROUND", "ALL") 
            Buffer_1KM = arcpy.Buffer_analysis(perimeter, rootpath + 'BufferLayers\\' + fr + "_" + yr + "_" + str(JD) +"_1Km.shp",
                                  "1000 METERS", "OUTSIDE_ONLY", "ROUND", "ALL")     
            Buffer_5KM = arcpy.Buffer_analysis(perimeter, rootpath + 'BufferLayers\\' + fr + "_" + yr + "_" + str(JD) +"_5Km.shp",
                                  "5000 METERS", "OUTSIDE_ONLY", "ROUND", "ALL")    
            arcpy.Merge_management([Buffer_200M, Buffer_1KM, Buffer_5KM], buff_fireFolder + "\\" + outName)
    except: 
        print("error " + fr + "_" + str(JD))
        errorFires.append(fr + "_" + str(JD) + "_" + str(index))
        continue 

184 Corralcomplexcorral 229
error Corralcomplexcorral_229
185 Corralcomplexcorral 230
186 Corralcomplexcorral 231
187 Corralcomplexcorral 232
188 Deluz 278
189 Deluz 279
190 Falls 217
193 General 146
194 General 147
195 Grand 135
196 Grand 136
197 Hathaway 160
198 Hathaway 161
199 Hathaway 162
200 Mccabe 326
201 Mccabe 327
202 Morgan 251
203 Morgan 252
204 Mountain 196
205 Mountain 197
206 Panther 115
207 Panther 116
208 Panther 117
209 Panther 118
210 Panther 119
211 Panther 120
212 Panther 121
213 Powerhouse 150
214 Powerhouse 151
215 Powerhouse 152
216 Rim 230
217 Rim 231
218 Rim 232
219 Rim 233
220 Rim 234
221 Rim 235
222 Rim 236
223 Rim 237
224 Rim 238
225 Rim 239
226 Rim 240
227 Rim 241
228 Rim 242
229 Rim 243
230 Salmonrivercomplexboulder 212
231 Salmonrivercomplexboulder 213
232 Salmonrivercomplexboulder 214
233 Salmonrivercomplexboulder 215
234 Salmonrivercomplexboulder 216
235 Salmonrivercomplexboulder 217
236 Salmonrivercomplexboulder 218
237 Salmonrivercomplexboulder 219
23

617 Forkcomplexpeak 222
618 Forkcomplexpeak 223
619 Forkcomplexrail 212
620 Forkcomplexrail 213
621 Forkcomplexrail 214
622 Forkcomplexrail 215
623 Forkcomplexrail 216
624 Forkcomplexrail 217
625 Forkcomplexrail 218
626 Forkcomplexrail 219
627 Forkcomplexrail 220
628 Forkcomplexrail 221
629 Forkcomplexrail 222
630 Forkcomplexrail 223
631 Forkcomplexshiell 212
632 Forkcomplexshiell 213
633 Forkcomplexshiell 214
634 Forkcomplexshiell 215
635 Forkcomplexshiell 216
636 Forkcomplexshiell 217
637 Forkcomplexshiell 218
638 Forkcomplexshiell 219
639 Forkcomplexshiell 220
640 Forkcomplexshiell 221
641 Forkcomplexshiell 222
642 Forkcomplexshiell 223
643 Forkcomplexshiell 224
644 Forkcomplexshiell 225
645 Forkcomplexshiell 226
646 Forkcomplexshiell 227
647 Frog 211
648 Frog 212
649 Frog 213
650 Frog 214
651 Gasquetcomplexbear 214
652 Gasquetcomplexbear 215
653 Gasquetcomplexbear 216
654 Gasquetcomplexbear 217
655 Gasquetcomplexbear 218
656 Gasquetcomplexbear 219
657 Gasquetcomplexbear 220
658 Gas

971 Chimneyslu 232
972 Chimneyslu 233
973 Chimneyslu 234
974 Chimneyslu 235
975 Clayton 226
976 Cold 215
977 Coleman 156
978 Coleman 157
979 Curry 183
980 Deer 183
981 Deer 184
982 Erskine 175
983 Erskine 176
984 Gap 240
985 Gap 241
986 Gap 242
987 Gap 243
988 Gap 244
989 Gap 245
990 Gap 246
991 Gap 247
992 Gap 248
993 Goose 212
994 Goose 213
995 Hidden 313
996 Hidden 314
997 Hidden 315
998 Hidden 316
999 Hidden 317
1000 Hidden 318
1001 Hidden 319
1002 Hidden 320
1003 Hidden 321
1004 Hidden 322
1005 Hidden 323
1006 Jacobson 294
1007 Jacobson 295
1008 Jacobson 296
1009 Lakes 252
1010 Lakes 253
1011 Lakes 254
1012 Lakes 255
1013 Lakes 256
1014 Lakes 257
1015 Lakes 258
1016 Lakes 259
1017 Lakes 260
1018 Lakes 261
1019 Lakes 262
1020 Lakes 263
1021 Lakes 264
1022 Loma 270
1023 Marshes 270
1024 Marshes 271
1025 Mineral 221
1026 Mineral 222
1027 Owensriver 261
1028 Pilot 220
1029 Pine 182
1030 Pine 183
1031 Pine 184
1032 Pine 185
1033 Pine 186
1034 Pony 159
1035 Pony 160
1036 Pony 161
1037 P

1410 Lion 273
1411 Lion 274
1412 Lion 275
1413 Lion 276
1414 Lion 277
1415 Lion 278
1416 Lion 279
1417 Lion 280
1418 Lion 281
1419 Lion 282
1420 Lion 283
1421 Lion 284
1422 Lion 285
1423 Manzanita 177
1424 Manzanita 178
1425 Mccormick 239
1426 Mccormick 240
1427 Mccormick 241
1428 Mccormick 242
1429 Mccormick 243
1430 Mccormick 244
1431 Millercomplexabney 269
1432 Millercomplexabney 270
1433 Millercomplexabney 271
1434 Millercomplexabney 272
1435 Millercomplexabney 273
1436 Millercomplexabney 274
1437 Millercomplexabney 275
1438 Millercomplexabney 276
1439 Millercomplexabney 277
1440 Millercomplexabney 278
1441 Millercomplexabney 279
1442 Millercomplexabney 280
1443 Millercomplexabney 281
1444 Millercomplexabney 282
1445 Millercomplexabney 283
1446 Millercomplexabney 284
1447 Millercomplexabney 285
1448 Millercomplexabney 286
1449 Minerva5 210
1450 Minerva5 211
1451 Minerva5 212
1452 Minerva5 213
1453 Minerva5 214
1454 Minerva5 215
1455 Minerva5 216
1456 Mission 246
1457 Mission 247
14

1829 Lions 187
1830 Lions 188
1831 Lions 189
1832 Lions 190
1833 Lions 191
1834 Lions 192
1835 Lions 193
1836 Lions 194
1837 Lions 195
1838 Lions 196
1839 Lions 197
1840 Lions 198
1841 Lions 199
1842 Lions 200
1843 Lions 201
1844 Lions 202
1845 Lions 203
1846 Lions 204
1847 Lions 205
1848 Lions 206
1849 Lions 207
1850 Lions 208
1851 Lions 209
1852 Lions 210
1853 Lions 211
1854 Lions 212
1855 Lions 213
1856 Lions 214
1857 Lions 215
1858 Lions 216
1859 Lions 217
1860 Lions 218
1861 Lions 219
1862 Lions 220
1863 Lions 221
1864 Lions 222
1865 Lions 223
1866 Lions 224
1867 Lions 225
1868 Lions 226
1869 Lions 227
1870 Lions 228
1871 Mendocinocomplexranch 208
1872 Mendocinocomplexranch 209
1873 Mendocinocomplexranch 210
1874 Mendocinocomplexranch 211
1875 Mendocinocomplexranch 212
1876 Mendocinocomplexranch 213
1877 Mendocinocomplexranch 214
1878 Mendocinocomplexranch 215
1879 Mendocinocomplexranch 216
1880 Mendocinocomplexranch 217
1881 Mendocinocomplexranch 218
1882 Mendocinocomplexranch 21

In [54]:
BUFFLIST = lstFiles(r"F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff", '.shp')
len(BUFFLIST)

2030

In [ ]:
'/Users/erica/Downloads/20120101_burning_index_g.tif'
gdalwarp -ot Int16 -r bilinear -tr 0.00892857142857143 0.00892857142857143 -s_srs EPSG:4326 /home/escaduto/GridMet/Extracted_Clipped/burning_index_g/20120101_burning_index_g.tif /home/escaduto/GridMet/Resampled/burning_index_g/20120101_burning_index_g.tif

In [5]:
import arcpy
from arcpy import env
from arcpy.sa import *

# Calculate drainage flow 

# Set local variables
inSurfaceRaster = r"F:\DriversFireProject\Topography\DEM\DEM_CA.tif"

# Execute FlowDirection
outFlowDirection = FlowDirection(inSurfaceRaster, "NORMAL")

outFlowAccumulation = FlowAccumulation(outFlowDirection)
outFlowAccumulation.save(r"F:\DriversFireProject\Topography\Flow_Direction\outFlow.tif")

In [9]:
BUFFLIST = lstFiles(r"F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff", '.shp')
BUFFLIST

['F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\MergedBuff\\2012\\Bagleycomplex\\Bagleycomplex_2012_0_NAT.shp',
 'F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\MergedBuff\\2012\\Bagleycomplex\\Bagleycomplex_2012_231_NAT.shp',
 'F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\MergedBuff\\2012\\Bagleycomplex\\Bagleycomplex_2012_232_NAT.shp',
 'F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\MergedBuff\\2012\\Bagleycomplex\\Bagleycomplex_2012_233_NAT.shp',
 'F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\MergedBuff\\2012\\Bagleycomplex\\Bagleycomplex_2012_234_NAT.shp',
 'F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\MergedBuff\\2012\\Bagleycomplex\\Bagleycomplex_2012_235_NAT.shp',
 'F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\MergedBuff\\2012\\Bagleycomplex\\Bagleycomplex_2012_236_NAT.shp',
 'F:\\DriversFireProject\\NaturalNeighborResults\\Daily\\MergedBuff\\2012\\Bagleycomplex\\Bagleycomplex_2012_237_NAT.shp',
 'F:\\DriversFireP

In [11]:
# add julian day and buff ID 
# key JD_ID
# Merge by fire 

fire = [] 
pth = []
year = []
julianDay = []

for buff in BUFFLIST: 
    nm = buff.split("\\")
    name = nm[-1]
    nmm = name.split("_")
    fr = nmm[0]
    yr = nmm[1]
    jd = nmm[2]
    julianDay.append(jd)
    year.append(yr)
    fire.append(fr)
    pth.append(buff)
    
    print(name, yr)
    
    arcpy.AddField_management(buff, 'JD_ID', "TEXT")
    with arcpy.da.UpdateCursor(buff, ["FID", "JD_ID"]) as cursor:
        for row in cursor: 
            #print(int(row[0]))
            row[1] = str(jd) + "_" + str(row[0])
            cursor.updateRow(row)


BuffDF = pd.DataFrame({'Fire': fire, 'Path': pth, 'julianDay': julianDay, 'Year':year})

Bagleycomplex_2012_0_NAT.shp 2012
Bagleycomplex_2012_231_NAT.shp 2012
Bagleycomplex_2012_232_NAT.shp 2012
Bagleycomplex_2012_233_NAT.shp 2012
Bagleycomplex_2012_234_NAT.shp 2012
Bagleycomplex_2012_235_NAT.shp 2012
Bagleycomplex_2012_236_NAT.shp 2012
Bagleycomplex_2012_237_NAT.shp 2012
Bagleycomplex_2012_238_NAT.shp 2012
Bagleycomplex_2012_239_NAT.shp 2012
Bagleycomplex_2012_240_NAT.shp 2012
Bagleycomplex_2012_241_NAT.shp 2012
Chihuahua2_2012_0_NAT.shp 2012
Chihuahua2_2012_222_NAT.shp 2012
Chihuahua2_2012_223_NAT.shp 2012
Chips_2012_0_NAT.shp 2012
Chips_2012_210_NAT.shp 2012
Chips_2012_211_NAT.shp 2012
Chips_2012_212_NAT.shp 2012
Chips_2012_213_NAT.shp 2012
Chips_2012_214_NAT.shp 2012
Chips_2012_215_NAT.shp 2012
Chips_2012_216_NAT.shp 2012
Chips_2012_217_NAT.shp 2012
Chips_2012_218_NAT.shp 2012
Chips_2012_219_NAT.shp 2012
Chips_2012_220_NAT.shp 2012
Chips_2012_221_NAT.shp 2012
Chips_2012_222_NAT.shp 2012
Chips_2012_223_NAT.shp 2012
Chips_2012_224_NAT.shp 2012
Chips_2012_225_NAT.shp 2012

Salmonrivercomplexboulder_2013_222_NAT.shp 2013
Salmonrivercomplexboulder_2013_223_NAT.shp 2013
Salmonrivercomplexboulder_2013_224_NAT.shp 2013
Salmonrivercomplexshelly_2013_0_NAT.shp 2013
Salmonrivercomplexshelly_2013_213_NAT.shp 2013
Salmonrivercomplexshelly_2013_214_NAT.shp 2013
Salmonrivercomplexshelly_2013_215_NAT.shp 2013
Salmonrivercomplexshelly_2013_216_NAT.shp 2013
Salmonrivercomplexshelly_2013_217_NAT.shp 2013
Salmonrivercomplexshelly_2013_218_NAT.shp 2013
Silver_2013_0_NAT.shp 2013
Silver_2013_219_NAT.shp 2013
Swedes_2013_0_NAT.shp 2013
Swedes_2013_228_NAT.shp 2013
White_2013_0_NAT.shp 2013
White_2013_147_NAT.shp 2013
White_2013_148_NAT.shp 2013
Basilonecomplexcombat_2014_0_NAT.shp 2014
Basilonecomplexcombat_2014_136_NAT.shp 2014
Basilonecomplexpulgas_2014_0_NAT.shp 2014
Basilonecomplexpulgas_2014_134_NAT.shp 2014
Basilonecomplexpulgas_2014_135_NAT.shp 2014
Basilonecomplextomahawk_2014_0_NAT.shp 2014
Basilonecomplextomahawk_2014_134_NAT.shp 2014
Basilonecomplextomahawk_2014_

Happycampcomplexfryingpanfaulkstein_2014_241_NAT.shp 2014
Happycampcomplexfryingpanfaulkstein_2014_242_NAT.shp 2014
Happycampcomplexfryingpanfaulkstein_2014_243_NAT.shp 2014
Happycampcomplexfryingpanfaulkstein_2014_244_NAT.shp 2014
Happycampcomplexfryingpanfaulkstein_2014_245_NAT.shp 2014
Happycampcomplexfryingpanfaulkstein_2014_246_NAT.shp 2014
Happycampcomplexfryingpanfaulkstein_2014_247_NAT.shp 2014
Happycampcomplexfryingpanfaulkstein_2014_248_NAT.shp 2014
Happycampcomplexfryingpanfaulkstein_2014_249_NAT.shp 2014
Happycampcomplexfryingpanfaulkstein_2014_250_NAT.shp 2014
Julycomplexlog_2014_0_NAT.shp 2014
Julycomplexlog_2014_211_NAT.shp 2014
Julycomplexlog_2014_212_NAT.shp 2014
Julycomplexlog_2014_213_NAT.shp 2014
Julycomplexlog_2014_214_NAT.shp 2014
Julycomplexlog_2014_215_NAT.shp 2014
Julycomplexlog_2014_216_NAT.shp 2014
Julycomplexlog_2014_217_NAT.shp 2014
Julycomplexlog_2014_218_NAT.shp 2014
Julycomplexlog_2014_219_NAT.shp 2014
Julycomplexlog_2014_220_NAT.shp 2014
Julycomplexlog_

Gasquetcomplexcoon_2015_222_NAT.shp 2015
Gasquetcomplexcoon_2015_223_NAT.shp 2015
Gasquetcomplexcoon_2015_224_NAT.shp 2015
Gasquetcomplexcoon_2015_225_NAT.shp 2015
Gasquetcomplexcoon_2015_226_NAT.shp 2015
Gasquetcomplexcoon_2015_227_NAT.shp 2015
Gasquetcomplexcoon_2015_228_NAT.shp 2015
Gasquetcomplexcoon_2015_229_NAT.shp 2015
Gasquetcomplexcoon_2015_230_NAT.shp 2015
Gasquetcomplexcoon_2015_231_NAT.shp 2015
Gasquetcomplexpeak_2015_0_NAT.shp 2015
Gasquetcomplexpeak_2015_213_NAT.shp 2015
Gasquetcomplexpeak_2015_214_NAT.shp 2015
Gasquetcomplexpeak_2015_215_NAT.shp 2015
Gasquetcomplexpeak_2015_216_NAT.shp 2015
Gasquetcomplexpeak_2015_217_NAT.shp 2015
Gasquetcomplexpeak_2015_218_NAT.shp 2015
Gasquetcomplexpeak_2015_219_NAT.shp 2015
Gasquetcomplexpeak_2015_220_NAT.shp 2015
Gasquetcomplexpeak_2015_221_NAT.shp 2015
Gasquetcomplexpeak_2015_222_NAT.shp 2015
Gasquetcomplexpeak_2015_223_NAT.shp 2015
Gasquetcomplexpeak_2015_224_NAT.shp 2015
Gasquetcomplexpeak_2015_225_NAT.shp 2015
Gasquetcomplexpeak

Solimar_2015_0_NAT.shp 2015
Solimar_2015_360_NAT.shp 2015
Southcomplexcastle_2015_0_NAT.shp 2015
Southcomplexcastle_2015_212_NAT.shp 2015
Southcomplexcastle_2015_213_NAT.shp 2015
Southcomplexcastle_2015_214_NAT.shp 2015
Southcomplexcastle_2015_215_NAT.shp 2015
Southcomplexcastle_2015_216_NAT.shp 2015
Southcomplexcastle_2015_217_NAT.shp 2015
Southcomplexcastle_2015_218_NAT.shp 2015
Southcomplexcastle_2015_219_NAT.shp 2015
Southcomplexcastle_2015_220_NAT.shp 2015
Southcomplexcastle_2015_221_NAT.shp 2015
Southcomplexcastle_2015_222_NAT.shp 2015
Southcomplexcastle_2015_223_NAT.shp 2015
Southcomplexcastle_2015_224_NAT.shp 2015
Southcomplexcastle_2015_225_NAT.shp 2015
Southcomplexpattison_2015_0_NAT.shp 2015
Southcomplexpattison_2015_212_NAT.shp 2015
Southcomplexpattison_2015_213_NAT.shp 2015
Southcomplexpattison_2015_214_NAT.shp 2015
Southcomplexpattison_2015_215_NAT.shp 2015
Southcomplexpattison_2015_216_NAT.shp 2015
Southcomplexpattison_2015_217_NAT.shp 2015
Southcomplexpattison_2015_218_

Soupcomplexsoup2_2016_262_NAT.shp 2016
Trailhead_2016_0_NAT.shp 2016
Trailhead_2016_180_NAT.shp 2016
Trailhead_2016_181_NAT.shp 2016
Trailhead_2016_182_NAT.shp 2016
Trailhead_2016_183_NAT.shp 2016
Willard_2016_0_NAT.shp 2016
Willard_2016_255_NAT.shp 2016
Willard_2016_256_NAT.shp 2016
Adobe_2017_0_NAT.shp 2017
Adobe_2017_281_NAT.shp 2017
Adobe_2017_282_NAT.shp 2017
Alamo_2017_0_NAT.shp 2017
Alamo_2017_188_NAT.shp 2017
Atlas_2017_0_NAT.shp 2017
Atlas_2017_281_NAT.shp 2017
Atlas_2017_282_NAT.shp 2017
Berry_2017_0_NAT.shp 2017
Berry_2017_255_NAT.shp 2017
Berry_2017_256_NAT.shp 2017
Berry_2017_257_NAT.shp 2017
Berry_2017_258_NAT.shp 2017
Buck_2017_0_NAT.shp 2017
Buck_2017_256_NAT.shp 2017
Buck_2017_257_NAT.shp 2017
Buck_2017_258_NAT.shp 2017
Buck_2017_259_NAT.shp 2017
Buck_2017_260_NAT.shp 2017
Buck_2017_261_NAT.shp 2017
Buck_2017_262_NAT.shp 2017
Buck_2017_263_NAT.shp 2017
Buck_2017_264_NAT.shp 2017
Buck_2017_265_NAT.shp 2017
Buck_2017_266_NAT.shp 2017
Buck_2017_267_NAT.shp 2017
Buck_2017_

Empire_2017_254_NAT.shp 2017
Empire_2017_255_NAT.shp 2017
Empire_2017_256_NAT.shp 2017
Empire_2017_257_NAT.shp 2017
Empire_2017_258_NAT.shp 2017
Empire_2017_259_NAT.shp 2017
Empire_2017_260_NAT.shp 2017
Empire_2017_261_NAT.shp 2017
Empire_2017_262_NAT.shp 2017
Empire_2017_263_NAT.shp 2017
Empire_2017_264_NAT.shp 2017
Empire_2017_265_NAT.shp 2017
Empire_2017_266_NAT.shp 2017
Empire_2017_267_NAT.shp 2017
Empire_2017_268_NAT.shp 2017
Empire_2017_269_NAT.shp 2017
Empire_2017_270_NAT.shp 2017
Empire_2017_271_NAT.shp 2017
Empire_2017_272_NAT.shp 2017
Empire_2017_273_NAT.shp 2017
Empire_2017_274_NAT.shp 2017
Empire_2017_275_NAT.shp 2017
Empire_2017_276_NAT.shp 2017
Empire_2017_277_NAT.shp 2017
Empire_2017_278_NAT.shp 2017
Empire_2017_279_NAT.shp 2017
Gardensqf_2017_0_NAT.shp 2017
Gardensqf_2017_211_NAT.shp 2017
Gardensqf_2017_212_NAT.shp 2017
Garza_2017_0_NAT.shp 2017
Garza_2017_190_NAT.shp 2017
Garza_2017_191_NAT.shp 2017
Garza_2017_192_NAT.shp 2017
Garza_2017_193_NAT.shp 2017
Gate_2017_0_NA

Pocket_2017_284_NAT.shp 2017
Pocket_2017_285_NAT.shp 2017
Pocket_2017_286_NAT.shp 2017
Pocket_2017_287_NAT.shp 2017
Pocket_2017_288_NAT.shp 2017
Pocket_2017_289_NAT.shp 2017
Ponderosa_2017_0_NAT.shp 2017
Ponderosa_2017_241_NAT.shp 2017
Prescott_2017_0_NAT.shp 2017
Prescott_2017_220_NAT.shp 2017
Prescott_2017_221_NAT.shp 2017
Prescott_2017_222_NAT.shp 2017
Prescott_2017_223_NAT.shp 2017
Prescott_2017_224_NAT.shp 2017
Prescott_2017_225_NAT.shp 2017
Railroad_2017_0_NAT.shp 2017
Railroad_2017_241_NAT.shp 2017
Railroad_2017_242_NAT.shp 2017
Railroad_2017_243_NAT.shp 2017
Railroad_2017_244_NAT.shp 2017
Railroad_2017_245_NAT.shp 2017
Railroad_2017_246_NAT.shp 2017
Ruthcomplexdutchman_2017_0_NAT.shp 2017
Ruthcomplexdutchman_2017_220_NAT.shp 2017
Ruthcomplexdutchman_2017_221_NAT.shp 2017
Ruthcomplexwillie_2017_0_NAT.shp 2017
Ruthcomplexwillie_2017_222_NAT.shp 2017
Ruthcomplexwillie_2017_223_NAT.shp 2017
Ruthcomplexwillie_2017_224_NAT.shp 2017
Rye_2017_0_NAT.shp 2017
Rye_2017_339_NAT.shp 2017
Sa

South_2019_254_NAT.shp 2019
South_2019_255_NAT.shp 2019
Springs_2019_0_NAT.shp 2019
Springs_2019_214_NAT.shp 2019
Springs_2019_215_NAT.shp 2019
Springs_2019_216_NAT.shp 2019
Springs_2019_217_NAT.shp 2019
Springs_2019_218_NAT.shp 2019
Springs_2019_219_NAT.shp 2019
Springs_2019_220_NAT.shp 2019
Springs_2019_221_NAT.shp 2019
Springs_2019_222_NAT.shp 2019
Springs_2019_223_NAT.shp 2019
Springs_2019_224_NAT.shp 2019
Springs_2019_225_NAT.shp 2019
Springs_2019_226_NAT.shp 2019
Springs_2019_227_NAT.shp 2019
Springs_2019_228_NAT.shp 2019
Springs_2019_229_NAT.shp 2019
Springs_2019_230_NAT.shp 2019
Springs_2019_231_NAT.shp 2019
Springs_2019_232_NAT.shp 2019
Springs_2019_233_NAT.shp 2019
Springs_2019_234_NAT.shp 2019
Springs_2019_235_NAT.shp 2019
Springs_2019_236_NAT.shp 2019
Springs_2019_237_NAT.shp 2019
Springs_2019_238_NAT.shp 2019
Springs_2019_239_NAT.shp 2019
Tucker_2019_0_NAT.shp 2019
Tucker_2019_209_NAT.shp 2019
Walker_2019_0_NAT.shp 2019
Walker_2019_247_NAT.shp 2019
Walker_2019_248_NAT.shp 

In [22]:
BuffDF

grouped = BuffDF.groupby(['Fire', 'Year'])['Path'].apply(list).reset_index()
grouped

,Fire,Year,Path
0,Adobe,2017,[F:\DriversFireProject\NaturalNeighborResults\...
1,Alamo,2017,[F:\DriversFireProject\NaturalNeighborResults\...
2,Alder,2018,[F:\DriversFireProject\NaturalNeighborResults\...
3,American,2013,[F:\DriversFireProject\NaturalNeighborResults\...
4,Aspen,2013,[F:\DriversFireProject\NaturalNeighborResults\...
...,...,...,...
252,Winter,2017,[F:\DriversFireProject\NaturalNeighborResults\...
253,Woolsey,2018,[F:\DriversFireProject\NaturalNeighborResults\...
254,Wragg,2015,[F:\DriversFireProject\NaturalNeighborResults\...
255,Wye,2012,[F:\DriversFireProject\NaturalNeighborResults\...


In [24]:
outpath = r'F:\DriversFireProject\NaturalNeighborResults\Daily\MergedByFire\\'

for index, row in grouped.iterrows(): 
    pth = row.Path[0]
    fr = row.Fire
    yr = row.Year
    print(row.Path[0], fr, yr) 
    '''fieldMappings = arcpy.FieldMappings()
    fieldMappings.addTable(row.Path[0])
    for field in fieldMappings.fields:
        if field.name not in ["Id", "JD_ID"]:
            fieldMappings.removeFieldMap(fieldMappings.findFieldMapIndex(field.name))'''
    yrfolder = outpath + str(yr)
    if not os.path.exists(yrfolder):
        os.makedirs(yrfolder)
    fireFolder = yrfolder + "\\" + fr
    if not os.path.exists(fireFolder):
        os.makedirs(fireFolder)
    out = fireFolder + "\\"  + fr + "_" + yr + "_Buff_Merged.shp"        
    arcpy.Merge_management(row.Path, out, fieldMappings)

F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2017\Adobe\Adobe_2017_0_NAT.shp Adobe 2017
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2017\Alamo\Alamo_2017_0_NAT.shp Alamo 2017
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2018\Alder\Alder_2018_0_NAT.shp Alder 2018
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2013\American\American_2013_0_NAT.shp American 2013
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2013\Aspen\Aspen_2013_0_NAT.shp Aspen 2013
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2017\Atlas\Atlas_2017_0_NAT.shp Atlas 2017
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2012\Bagleycomplex\Bagleycomplex_2012_0_NAT.shp Bagleycomplex 2012
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2014\Basilonecomplexcombat\Basilonecomplexcombat_2014_0_NAT.shp Basilonecomplexcombat 2014
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2014\Basilonecomp

F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2015\Forkcomplexbarker\Forkcomplexbarker_2015_0_NAT.shp Forkcomplexbarker 2015
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2015\Forkcomplexblue\Forkcomplexblue_2015_0_NAT.shp Forkcomplexblue 2015
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2015\Forkcomplexpeak\Forkcomplexpeak_2015_0_NAT.shp Forkcomplexpeak 2015
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2015\Forkcomplexrail\Forkcomplexrail_2015_0_NAT.shp Forkcomplexrail 2015
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2015\Forkcomplexshiell\Forkcomplexshiell_2015_0_NAT.shp Forkcomplexshiell 2015
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2012\Fortcomplex\Fortcomplex_2012_0_NAT.shp Fortcomplex 2012
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2014\French\French_2014_0_NAT.shp French 2014
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2015\Frog\Frog_2015_

F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2012\Mdflakecomplex\Mdflakecomplex_2012_0_NAT.shp Mdflakecomplex 2012
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2018\Mendocinocomplexranch\Mendocinocomplexranch_2018_0_NAT.shp Mendocinocomplexranch 2018
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2018\Mendocinocomplexriver\Mendocinocomplexriver_2018_0_NAT.shp Mendocinocomplexriver 2018
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2019\Middle\Middle_2019_0_NAT.shp Middle 2019
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2018\Millcreek1\Millcreek1_2018_0_NAT.shp Millcreek1 2018
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2017\Millercomplexabney\Millercomplexabney_2017_0_NAT.shp Millercomplexabney 2017
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2016\Mineral\Mineral_2016_0_NAT.shp Mineral 2016
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2017\Minerva5\Min

F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2012\Scotts\Scotts_2012_0_NAT.shp Scotts 2012
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2016\Sherpa\Sherpa_2016_0_NAT.shp Sherpa 2016
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2014\Shirley\Shirley_2014_0_NAT.shp Shirley 2014
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2012\Shockey\Shockey_2012_0_NAT.shp Shockey 2012
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2018\Shucreek\Shucreek_2018_0_NAT.shp Shucreek 2018
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2013\Silver\Silver_2013_0_NAT.shp Silver 2013
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2012\Sitescomplex\Sitescomplex_2012_0_NAT.shp Sitescomplex 2012
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2012\Sixteen\Sixteen_2012_0_NAT.shp Sixteen 2012
F:\DriversFireProject\NaturalNeighborResults\Daily\MergedBuff\2016\Slate\Slate_2016_0_NAT.shp Slate 201